## 상품정보 데이터

### json데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

import re
import warnings
import os
warnings.filterwarnings(action='ignore')
#pd.set_option('옵션 이름', 수정할 값)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

## items

In [ ]:
items = pd.read_json('./items.json')
items.head()

In [ ]:
items.info()

### 상품명 전처리

In [ ]:
def item_name(row):
    return row.split('\n')[0] # 줄바꿈이전 문자열만 남기기

items['name'] = items['name'].apply(item_name)
items.head()

### 상품명 중복확인

In [ ]:
dup_idx = items.loc[items['name'].duplicated()==True].index
items.drop(dup_idx, axis=0, inplace=True)
items['name'].duplicated().sum()


In [ ]:
# 인덱스 재설정
items.reset_index(drop=True, inplace=True)

In [ ]:
items.info()

### Item_id 부여

In [ ]:
items['item_id'] = [(5-len((str(i)))) * '0' + str(i) for i in list(items.index)]
items.head()

In [ ]:
items.tail()

In [ ]:
# items.to_json('../Load/items_prc.json', orient='records', indent=2, force_ascii=False) # 객체기준 레코드, 들여쓰기 적용, 한글깨짐 해결

In [ ]:
# # 리뷰 크롤링용 데이터 저장
# items[['item_id', 'link']].to_json('../Extract/items_link.json', orient='records', indent=2, force_ascii=False)

# review data

In [2]:
review = pd.read_json('./reviews.json')

# 불필요한 컬럼 삭제
review = review[['품번', '아이디', '별점', '키', '몸무게', '평소사이즈', '타입', '리뷰', '사이즈', '컬러', '색상', '타입2']] #, '색상-사이즈', '데님'
review.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   품번      3081 non-null   int64 
 1   아이디     3081 non-null   object
 2   별점      3081 non-null   int64 
 3   키       2298 non-null   object
 4   몸무게     2122 non-null   object
 5   평소사이즈   2191 non-null   object
 6   타입      2827 non-null   object
 7   리뷰      3081 non-null   object
 8   사이즈     48 non-null     object
 9   컬러      105 non-null    object
 10  색상      273 non-null    object
 11  타입2     68 non-null     object
dtypes: int64(2), object(10)
memory usage: 289.0+ KB


In [3]:
review.head()

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,컬러,색상,타입2
0,464,mg,5,161~165,None,None,None,아직 안입어봤지만 여름에 입기 좋을거같아요\n기본 디자인에 블랙앤화이트라 무난하고\...,NaN,블랙(black)-F,NaN,NaN
1,464,188592267,5,None,None,None,None,후기\n두께도 얇은편이고 니트느낌나는 재질이기도 하고 사이즈도 널럴해서 바람도 잘 ...,NaN,블랙(black)-F,NaN,NaN
2,464,363732161,5,156~160,51 ~ 55,66,None,완전 제서타일!!\n빨앗는데 달라지는 느낌 없구 됴아요!!\n가오리 핏 됴아용,NaN,블랙(black)-F,NaN,NaN
3,464,291515456,5,156~160,65~,XXL,None,넉넉하고 편함\n얆은 니트라 덥지도않아요\n컬러감좋아요.딸아이 입고 이쁘다맘에들어해요,NaN,크림(cream)-F,NaN,NaN
4,464,225725978,1,None,None,None,None,옷은 너무 맘에 들어요 근데..딱 두번 입었는데 폈어요ㅠㅠ 옷 험하게 입는 스타일도...,NaN,크림(cream)-F,NaN,NaN


In [3]:
# NaN -> None
review = review.replace({np.nan: None})

### 카디널리티 확인

In [5]:
for col in ['별점', '키', '몸무게', '평소사이즈']:
    print(col)
    print(review[col].unique())
    print('\n')

별점
[5 1 4 3 2]


키
['161~165' None '156~160' '166~170' '151~155' '171~175' '139이하' '176이상'
 '146~150']


몸무게
[None '51 ~ 55' '65~' '61 ~ 65' '56 ~ 60' '46 ~ 50' '~45']


평소사이즈
[None '66' 'XXL' '88' '77' '99' '55' 'L' 'M' 'XXXL' 'XL' '100' '44' 'S']




* 핫핑 사이즈 기준

<img src='./imgs/사이즈통일.png'>

In [4]:
# 평소 사이즈 수정
key1 = [str(i) for i in range(44, 100, 11)] +['100']
key2 = ['S', 'M', 'L', 'XL', 'XXL', 'XXXL']

def user_size(row):
    for i in range(1,6):
        if row == key1[i]:
            return str(i)
        elif row == key2[i]:
            return str(i)
        
review['평소사이즈'] = review['평소사이즈'].apply(user_size)
review['평소사이즈'].unique()

array([None, '2', '4', '3', '5', '1'], dtype=object)

In [5]:
# 리뷰 데이터 줄바꿈 -> 공백으로 대체
def review_comm(row):
    return re.sub('\n', ' ', row)

review['리뷰'] = review['리뷰'].apply(review_comm)
review.head()

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,컬러,색상,타입2
0,464,mg,5,161~165,None,None,None,아직 안입어봤지만 여름에 입기 좋을거같아요 기본 디자인에 블랙앤화이트라 무난하고 루...,None,블랙(black)-F,None,None
1,464,188592267,5,None,None,None,None,후기 두께도 얇은편이고 니트느낌나는 재질이기도 하고 사이즈도 널럴해서 바람도 잘 통...,None,블랙(black)-F,None,None
2,464,363732161,5,156~160,51 ~ 55,2,None,완전 제서타일!! 빨앗는데 달라지는 느낌 없구 됴아요!! 가오리 핏 됴아용,None,블랙(black)-F,None,None
3,464,291515456,5,156~160,65~,4,None,넉넉하고 편함 얆은 니트라 덥지도않아요 컬러감좋아요.딸아이 입고 이쁘다맘에들어해요,None,크림(cream)-F,None,None
4,464,225725978,1,None,None,None,None,옷은 너무 맘에 들어요 근데..딱 두번 입었는데 폈어요ㅠㅠ 옷 험하게 입는 스타일도...,None,크림(cream)-F,None,None


### ['사이즈', '컬러', '색상', '타입', '타입2', '색상-사이즈', '데님'] 컬럼 통일 필요 = **['사이즈: str', '색상: 한글']으로 통일**


<img src='./imgs/타입_해당컬럼.png' width=500 height=500/>

#### 각 컬럼별 카디널리티 줄이고 컬럼 통일

In [6]:
review.loc[review['사이즈']=='핑크']

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,컬러,색상,타입2
1709,525,핫핑-유림,5,161~165,46 ~ 50,1,None,봄에 필수로 소장해야하는 브이넥 반팔니트 요번에 시드니 촬영본 너무 예쁘지않나용.....,핑크,1,None,None


In [7]:
# 값 서로 바꾸기
idx = review.loc[review['사이즈']=='핑크'].index
review.loc[idx, '사이즈'], review.loc[idx, '컬러'] = review.loc[idx, '컬러'], review.loc[idx, '사이즈']
review.iloc[idx, :]

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,컬러,색상,타입2
1709,525,핫핑-유림,5,161~165,46 ~ 50,1,None,봄에 필수로 소장해야하는 브이넥 반팔니트 요번에 시드니 촬영본 너무 예쁘지않나용.....,1,핑크,None,None


In [10]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

사이즈
[None 'F' '1' 'free' '2' 'M']


컬러
['블랙(black)-F' '크림(cream)-F' None '그레이' '화이트' '아이보리' '스카이' '딥블루' '블랙'
 '베이지' '화이트-F' '핑크' '네이비-1' '아이보리-F' '크림(cream)' '차콜' '블' '바질' '오션블루'
 '화이트-1' '그린' '화이트(white)' '블랙(black)' '블랙(black)-1' '오트밀']


색상
[None '블랙' '화이트' '퍼플' '화이트(white)-F' '연그린(light green)-F' '그린(green)-F'
 '그린' '블랙(black)-F' '차콜(charcoal)-F' '연보라(light purple)-F' '네이비(navy)-F'
 '브라운(brown)-F' '핑크(pink)-F' '소라(sora)-F' '코발트(covalt)-F' '브라운' '아이보리']


타입
[None '오픈카라' '핑크{pink)-F' '네이비(navy)-F' '백염(W.salt)-F' '화이트(white)-F'
 '그린(green)-F' '블랙(black)-3' '아이보리(ivory)-F' '그레이(grey)-F' '퍼플(purple)-F'
 '백메란지(W.melange)-F' '딥블루(D.blue)-F' '블랙(black)-2' '화이트(white)-1'
 '화이트(white)-3' '차콜(charcoal)-F' '포레스트그린(f.green)-3' '블랙(black)-1'
 '화이트(white)-2' '포레스트그린(f.green)-2' '포레스트그린(f.green)-1' '멜란지그레이(m.grey)-1'
 '멜란지그레이(m.grey)-2' '블랙(black)-F' '쿨블루(c.blue)-1' '쿨블루(c.blue)-2'
 '쿨블루(c.blue)-3' '포그블루(F.blue)-3' '포그블루(F.blue)-2' '포그블루(F.blue)-1'
 '퓨어아이보리(p.ivory)-2' '퓨어아이보리(p.ivory)-1' '퓨어아이보리

#### 사이즈 있는경우 '색상-사이즈', 없는 경우 '색상' 형태로 값 통일

In [8]:
# 사이즈 컬럼
def size(row):
    if row in ['1', '2', '3', '4', 'F', '블랙(black)-2', '블랙 1']:
        return row
    elif row in ['1size', '미니-1', '1사이즈']:
        return re.sub('[^0-9]', '', row)
    elif row=='free':
        return 'F'
    elif row=='M':
        return '2'
    
review['사이즈'] = review['사이즈'].apply(size)

review.loc[review['사이즈']=='블랙 1', '사이즈'] = '블랙-1'

# 컬러 컬럼
def color(row):
    if row: # Nan => None
        if '_' in row:
            return row.split('_')[-1]
        elif '/' in row:
            return row.split('/')[-1]
        elif '미니-' in row:
            row = row.replace('미니-', '')
            return row
        elif ' ' in row and '(' not in row:
            return row.split()[-1]
        elif row == '블':
            return '블루'
        else:
            return row
    
review['컬러'] = review['컬러'].apply(color)


# 타입 컬럼
def type_prc(row):
    if not row: # None값 제외
        pass
    elif '. ' in row: 
        return row.split('-')[-1]
    elif ' [예약판매]' in row:
        row = row.replace(' [예약판매]', '')
        return row
    elif '(반팔)' in row:
        row = row.replace('(반팔)', '')
        return row
    elif '_' in row:
        return row.split('_')[-1]
    elif row[0].isdigit() and ' ' not in row: # return T/F
        return None
    elif row[0].isdigit() and ' ' in row:
        return row.split('-')[-1]
    elif '(' not in row: # 타입과 무관한값은 ()문자가 없음
        return None
    else:
        return row

review['타입'] = review['타입'].apply(type_prc)
    
# 데님, 색상-사이즈 컬럼
def denim_color(row):
    if type(row) == str:
        if '예약' in row:
            return row.split(' ')[0]
        else:
            return row

# review['데님'] = review['데님'].apply(denim_color)
# review['색상-사이즈'] = review['색상-사이즈'].apply(denim_color)
review['타입2'] = review['타입2'].apply(denim_color)

In [9]:
# 카디널리티 확인
cols = ['사이즈', '컬러', '색상', '타입', '타입2'] #, '색상-사이즈', '데님'
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

사이즈
[None 'F' '1' '2']


컬러
['블랙(black)-F' '크림(cream)-F' None '그레이' '화이트' '아이보리' '스카이' '딥블루' '블랙'
 '베이지' '화이트-F' '핑크' '네이비-1' '아이보리-F' '크림(cream)' '차콜' '블루' '바질' '오션블루'
 '화이트-1' '그린' '화이트(white)' '블랙(black)' '블랙(black)-1' '오트밀']


색상
[None '블랙' '화이트' '퍼플' '화이트(white)-F' '연그린(light green)-F' '그린(green)-F'
 '그린' '블랙(black)-F' '차콜(charcoal)-F' '연보라(light purple)-F' '네이비(navy)-F'
 '브라운(brown)-F' '핑크(pink)-F' '소라(sora)-F' '코발트(covalt)-F' '브라운' '아이보리']


타입
[None '네이비(navy)-F' '백염(W.salt)-F' '화이트(white)-F' '그린(green)-F'
 '블랙(black)-3' '아이보리(ivory)-F' '그레이(grey)-F' '퍼플(purple)-F'
 '백메란지(W.melange)-F' '딥블루(D.blue)-F' '블랙(black)-2' '화이트(white)-1'
 '화이트(white)-3' '차콜(charcoal)-F' '포레스트그린(f.green)-3' '블랙(black)-1'
 '화이트(white)-2' '포레스트그린(f.green)-2' '포레스트그린(f.green)-1' '멜란지그레이(m.grey)-1'
 '멜란지그레이(m.grey)-2' '블랙(black)-F' '쿨블루(c.blue)-1' '쿨블루(c.blue)-2'
 '쿨블루(c.blue)-3' '포그블루(F.blue)-3' '포그블루(F.blue)-2' '포그블루(F.blue)-1'
 '퓨어아이보리(p.ivory)-2' '퓨어아이보리(p.ivory)-1' '퓨어아이보리(p.ivory)-3'
 '멜란지그레이(m.grey)-

In [10]:
# 색상-사이즈 형태인 경우
    ## str.split()로 분리, 문자열 정규화로 데이터 수정
# 색상 데이터만 있는 경우
    ## 문자열 정규화만 수행
def col_merge(df, cols):
    def normalize(row):
        row[0] = re.sub('[^가-힣]','',row[0]) # 한글만 남기기
        return row

    # 사이즈 컬럼 별도 수행
    idx = df.loc[(review['사이즈']=='블랙-1') | (review['사이즈']=='블랙(black)-2')].index
    for i in idx:
        df.loc[i, ['색상', '사이즈']] = df.iloc[i]['사이즈'].split('-')
    for col in cols:
        idx = df.loc[df[col].notnull()==True].index
        for i in idx:
            row = df.iloc[i][col].split('-')
            if len(row)==2:
                df.loc[i, ['색상', '사이즈']] = normalize(row)
            else:
                df.loc[i, ['색상']] = normalize(row)[0]
    return df
cols = ['컬러', '타입', '타입2', '색상'] # , '색상-사이즈', '데님'
review = col_merge(review, cols)
review.sample(10)

,품번,아이디,별점,키,몸무게,평소사이즈,타입,리뷰,사이즈,컬러,색상,타입2
2458,542,255621357,5,None,None,None,그린(green)-2,스퀘어넥이여서 더 이쁩니다. 니트가 부드러워서 더 좋아요.,2,None,그린,None
450,492,6270209,5,166~170,65~,3,아이보리(ivory)-F,러블리 귀요미 담당이에요 제가 유치원에서 일하고 있는데 저희 아이들이 저보고 선생님...,F,None,아이보리,None
1930,530,aaa6,5,156~160,61 ~ 65,2,화이트(white)-3,배송은 조금 느렸어요~ 입기전엔 생각보다 짧아서 반품해야겠다 생각했는데 입으니까 핏...,3,None,화이트,None
1473,517,164644101,5,156~160,65~,4,블랙(black)-F,너무 덥지 않고 편하게 입기 좋아요 비침 없어서 좋습니다 한여름엔 좀 더울거 같아요,F,None,블랙,None
1127,507,dladkfkd4,5,161~165,61 ~ 65,2,None,가성비 대박 ㅋㅋㅋㅋ또살예정이에요 부들부들해서 촉감도 좋아요 다른색깔로도 재구매 ㄱㄱㄱ,None,None,None,None
1617,521,abetter3,5,166~170,65~,5,None,사이즈 여유롭게 잘 맞고요 치마든 바지든 코디하기 좋아서 출근룩 유니폼 처럼 입고 다녀요,F,None,블랙,블랙(black)-F
2802,549,gina,5,161~165,65~,4,블랙(black)-3,옷 진짜 편해요!! 강추 강추 또 살거예요!! 적당히 벙벙하고 여유있어서 막입기 좋아요,3,None,블랙,None
1166,509,2233570,5,None,None,None,블랙(black)-2,박스 스타일은 아니지만 그래도 시원한 소재라서 여름에 잘 입을 수 있을 거 같아요....,2,None,블랙,None
2314,539,121253582,5,156~160,65~,3,오트밀(oatmeal)-F,색별로 쟁였습니다 요즘날씨에 패딩조끼랑 입으면 딱이에요,F,None,오트밀,None
1476,517,4781732,5,166~170,65~,4,블랙(black)-F,기본으로 입기 좋아요~ 두께가 좀 있구 사이즈 좀 커요~ 그래두 데님에 코디하기 좋아요,F,None,블랙,None


* 카디널리티 확인

In [11]:
cols = ['사이즈', '색상']
for col in cols:
    print(col)
    print(review[col].unique())
    print('\n')

사이즈
['F' None '3' '2' '1' '4' 'M' 'L']


색상
['블랙' '크림' '그레이' None '네이비' '백염' '화이트' '그린' '아이보리' '스카이' '퍼플' '백메란지' '딥블루'
 '차콜' '포레스트그린' '멜란지그레이' '베이지' '쿨블루' '포그블루' '퓨어아이보리' '오트베이지' '핑크' '데이지옐로우'
 '샌드베이지' '진핑크' '옐로우' '메란지' '소라' '카키' '연그린' '연보라' '브라운' '코발트' '연베이지' '연카키'
 '딥옐로우' '블루' '버터' '연두' '바질' '오트밀' '라임' '더스트블루' '오션블루' '매트브라운' '브이넥블랙'
 '라이트블루' '백아이보리' '아쿠아블루' '오렌지' '소프트옐로우' '민트' '연핑크' '회소라' '모카' '뮤트블루']




In [14]:
def sel_size(row):
    size = ['S', 'M', 'L', 'XL', 'XL']
    size_dict = {size[i-1]: str(i) for i in range(1,6)}
    if row:
        if row.isdigit() or row=='F':
            return row
        else:
            return size_dict[row]

review['사이즈'] = review['사이즈'].apply(sel_size)
review['사이즈'].unique()

array(['F', None, '3', '2', '1', '4'], dtype=object)

In [15]:
# 현재까지 처리된 df 사본
review_prc = review.copy()

#### 사이즈 컬럼 추가 전처리

In [ ]:
# def last_size(row):
#     if row:
#         if row == 'L':
#             return '3'
#         elif row == 'B)':
#             return None
#         elif row == ' F':
#             return 'F'
#         else:
#             return row
# review_prc['사이즈'] = review_prc['사이즈'].apply(last_size)

# review_prc['사이즈'].unique()

            

* 나머지 컬럼은 삭제

In [16]:
# 통합 끝난 컬럼 제거 (색상, 사이즈)
review_prc.drop(columns=['컬러', '타입', '타입2'], inplace=True)# , '색상-사이즈', '데님'

### '별점' 컬럼 핫핑 직원 데이터는 제외

In [17]:
staffs = set()
p = re.compile('[ㄱ-힣]')
for name in review_prc['아이디'].unique():
    r = p.search(name)
    if r:
        staffs.add(name)
        
for s in staffs:
    review_prc.loc[review_prc['아이디'] == s, '별점'] = -1 # 분석에 사용하지 않음
          

In [18]:
review_prc['별점'].value_counts()

별점
 5    2782
 4     112
-1      92
 3      64
 1      18
 2      13
Name: count, dtype: int64

In [19]:
review_prc.to_json('../Load/reviews_prc_2.json', orient='records', indent=2, force_ascii=False)